# Feature
Run notebooks or their python exports from another notebook to add features and functions or use cells as templates. Tracks running notebooks(s) to prevent recursion and optionally create stats.

## Problem:
Code reuse doesn't work well across notebooks in Colab, and they're atypical in terms of software development. Repetitive code wastes time with copy/paste, duplication of work, and lack of code coherence.

## Constraints:
The feature shall:
- [ ] prevent recursion
- [ ] add `nb_name` and `running_nb` to `c`onfig
- [ ] ensure `features/` and `features/steps/` directories exist
- [ ] ensure `project/notebooks/` and `project/notebooks/templates/` directories exist
- [ ] provide `%%from_template {name}` cell magic with copy/paste output if existing version varies from saved
- [ ] provide `%%writerun {...}` cell magic to flexibly create files from cells
- [ ] provide `run_nb({name})` for intelligently running notebooks



# Imports

In [1]:
import os
import re
import requests
from IPython.core.magic import (register_cell_magic, register_line_magic)
from IPython import get_ipython

# standard first cell

In [24]:
# %%writerun {c['projects_dir']}{c['active_project']}/notebooks/templates/first_cell
try:
  c
except:
  c={'resources':{'jai':{}},'use_google_drive':True}
  !wget -qO epc.py https://maya.software/epc.txt
  import epc
  c = epc.do_setup(c)
#   %run {c.get('nb_dir')}notebooks_as_features.ipynb

# !cd {c['projects_dir']}{c['active_project']} && git pull

overwrote w /content/drive/MyDrive/Projects/jai/notebooks/templates/first_cell []


In [3]:
# after git clone/drive mount/etc make sure dirs exist
!mkdir -p {c['nb_dir']}
!mkdir -p {c['nb_dir']}templates
!mkdir -p {c['projects_dir']}{c['active_project']}/features
!mkdir -p {c['projects_dir']}{c['active_project']}/features/steps
# !ls {c['nb_dir']}
# !ls {c['projects_dir']}{c['active_project']}/notebooks
# !ls {c['projects_dir']}{c['active_project']}

bdd_testing_a_notebook.ipynb	jai_utils.ipynb
bootstrap_a_notebook.ipynb	setup_project.ipynb
demo_notebook.ipynb		templates
encrypted_private_config.ipynb	vite.ipynb
git_for_colab.ipynb
features  LICENSE  notebooks  README.md


# Get Notebook Data (including name!)
All the vital data on a notebook, including the file name and string to generate the URL.

In [5]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
def get_nb_data():
  # https://stackoverflow.com/questions/66175418/find-path-of-python-notebook-ipynb-when-running-it-with-google-colab
  session = requests.Session()
  retry = Retry(connect=1, backoff_factor=1)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  c['nb_ip'] = os.popen('hostname -I').read().strip()
  nb_address = 'http://'+c['nb_ip']+':9000/api/sessions' #172.28.0.2
  # return requests.get(nb_address).json()[0]
  return session.get(nb_address).json()[0]

In [6]:
# vital data for config
# c['nb'] = c['nb'] if 'nb' in c else get_nb_data()
if 'nb_name' not in c:
  nb_data = get_nb_data()
  c['nb_name'] = c['running_nb'] = nb_data['name'][:-6] # assuming w/running_nb; c might get cleared by anyone or this nb run not be in first_cell logic -- no way to tell which is which other than going back to defining it in every nb and changing on renames
  c['nb_path'] = nb_data['path'][7:]
  try:
    logger = epc.get_logger(c['nb_name']) # use if we have it
  except:
    'epc not loaded, new logger'
    import logging # extremely rare, do import here
    logger = logging.getLogger()
logger.info('main parent notebook: '+c['nb_name'])

2023-01-17 04:58:45 notebooks_as_features INFO     main parent notebook: notebooks_as_features


# Magic for Templates


Reuse of code is important and there aren't many tools for notebooks. You don't need much to mimic `%load` in Colab. 

## Create a Template 
```python
%%writerun -e -w {c['nb_dir']}templates/template_name
# handy cell content (-e executes and -w writes)
```
## Use a Template
```python
%%from_template template_name
#anything, it will complain if it doesn't match the template and print out the template code to copy into the cell
```
## Thoughts
Ideally code is cohesive and together in a module you can `import` or a notebook you can `%run` but there *are* legitimate use-cases where one-off templates beat [fetching and] including modules or running entire notebooks. This is why `%macro` exists, but in Colab those don't persisit after runtime restarts and to `%store` them you have to `%run` them to begin with.

In [7]:
@register_cell_magic
def writerun(line, cell):
  '''like %save but for a cell'''
  fname = line.split()[0]
  if not os.path.isdir(os.path.dirname(fname)):
    print('Directory does not [yet] exist:',os.path.dirname(fname))
  else:
    omode = 'a' if '-a' in line else 'w'
    overw = 'over' if os.path.isfile(fname) else ''
    with open(fname, omode) as f:
      f.write(cell)
    if not '-q' in line:
      print(overw+'wrote '+omode,fname,line.split()[1:])
  if '-e' in line:
    get_ipython().run_cell(cell)

@register_cell_magic
def from_template(line, cell):
  args = line.split()
  # remove active form #@param, a mismatch there is valid https://regex101.com/r/GfyE66/1
  p = re.compile(r'[a-zA-Z0-9_ *]{0,99}"? #@param', re.MULTILINE)
  print('template:',c['nb_dir']+'templates/'+args[0])
  if not os.path.isfile(c['nb_dir']+'templates/'+args[0]):
    print('Not found.')
    return False
  with open(c['nb_dir']+'templates/'+args[0], 'r') as f:
    contents = f.read()
  get_ipython().run_cell(cell)
  if p.sub('', cell).strip()!=p.sub('', contents).strip():
    # print(p.sub('', contents).strip())
    # print('CURRENT:')
    # print(p.sub('', cell).strip())
    print('Mismatch, template is:\n'+contents) #p.sub('', contents).strip())
    # import difflib
    # output_list = [li for li in difflib.ndiff(p.sub('', cell).strip(), p.sub('', contents).strip()) if li[0] != ' ']
    # print(output_list)

In [8]:
# c
# del c

# Replacement `%run` with Tracking
So we know what's running from where and don't end up in infinite loops.

In [9]:
def run_nb(name):
  print('run_nb(',name,')','nb_name:',c['nb_name'],'current running_nb set to:',c['running_nb'])
  logger = epc.get_logger(c['running_nb'])
  #TODO: can we catch nb_name->running-->RUNS ANOTHER? stop or give names back in order or..? array_pop, heh
  c['running_nb'] = name # c['running_nb'] if 'running_nb' in c else c['nb_name']
  if c['nb_name']!=c['running_nb']:
    print('running:',c['running_nb'])
    logger.info(c['nb_dir']+c['nb_name']+' --> '+c['nb_dir']+c['running_nb'])
    %run {c['nb_dir']}{c['running_nb']}.ipynb #TODO: non-ipython version .replace(' ','\ ') 
    c['running_nb']=c['nb_name'] # give name back
  else: print('recursion, not running',c['running_nb'])
  logger = epc.get_logger(c['nb_name'])

In [10]:
c

{'log_level': 10,
 'timezone': 'America/Anchorage',
 'use_google_drive': True,
 'epc': 'U2FsdGVkX1/xUi/hYJ+V9stP7WV+Y6fTtkJNZoDL8PX4mSWh/CuEAZUz0VI8r/BvrfQztrCI1rPmduBRSJsTLQeD9AKFNjHMNq15V1ArU7CvCVVXvuKgNsDS0aEN2hJoRMXiy4g7YzIB/b45e3vq8/DV/sFFml1GtWeTtIWLeA1c92UNrF8s3MjpvWhYB+GeD1OSQMcS3aoveW7YD8tSigzRkZQqwmoNrHlNdODtNeLq4idK4VsD9mauLGg/mWKitdfbqUi0yULYpg5pRxPMZaIy4sDFdGYbeaHYBaduf36Wr+ak3x6e7rvT7fF04EST3wv7FM54xAxZiyohgUHIyG+5oYBbvcWsKl74GZpoPYqUffxdnnH8VKURHTE28WaXwdsXwj0edbkX3cNcL/aCOdF9pfCcNPJFbRD0LM7E2sF9xYeqyNMhpnBMKWfXex7rEzj2ZRQbxlRy+f+476Vni/Qf2Ixcc/x5vCvAEJgCueD+loXol4oP3O4eDM6Q1FiPxMDOCYGxJ8TCFVb5795vqsmWsKLfa1pOqnPSJ2Id4ayiryeyxtDShRD8fTq0RplfXU7JMKnrCZJNq4CW+rDSRTOy4cHE0S3zy1EbFtPVP3Y4MCCKaWIHa/LVj9LDZSQvYSZO4xZb3Aw+fuvxqI/4Ow==',
 'resources': {'jai': {}},
 'active_project': 'jai',
 'projects_dir': '/content/drive/MyDrive/Projects/',
 'nb_dir': '/content/drive/MyDrive/Colab\\ Notebooks/',
 'cmds': [],
 'nb_ip': '172.28.0.2',
 'nb_name': 'notebooks_as_features',
 'running

In [ ]:
# manual set here, running_nb/run_nb() defined here can't be looped
if c.get('running_nb')=='notebooks_as_features':
  run_nb('git_for_colab')
  nb2repo()
  # git_ppush('first cell update')

2023-01-17 05:34:50 notebooks_as_features INFO     /content/drive/MyDrive/Colab\ Notebooks/notebooks_as_features --> /content/drive/MyDrive/Colab\ Notebooks/git_for_colab


run_nb( git_for_colab ) nb_name: notebooks_as_features current running_nb set to: notebooks_as_features
running: git_for_colab
Overwriting /root/.jupyter/nbdime_config.json
try to write
/content/drive/MyDrive/Projects/jai/notebooks/notebooks_as_features.ipynb


In [12]:
# !cd {c.get('projects_dir')}{c.get('active_project')} && git pull

In [13]:
# !cd {c['projects_dir']}{c['active_project']} && git a